<a href="https://colab.research.google.com/github/dolongbien/ML2018/blob/master/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import packages**

In [0]:
import math
import copy
from collections import Counter

**Utility classes**

In [0]:
class Node:
    def __init__(self, root):
        self.root = root
        self.children = []

    def addChild(self, children):
        self.children.append(children)


class Example:
    def __init__(self):
        self.attributeDict = {}
        self.label = None

    def addAtribute(self, attribute, value):
        self.attributeDict[attribute] = value

    def setLabel(self, label):
        self.label = label


class DecisionTree:

    def __init__(self):
        self.examples = []
        self.labels = []
        self.attributeList = []
        self.valueListDict = {}
        self.tree = None

    def load_data(self, file):
        header_line = file.readline().split()
        for name in header_line[:-1:]:  # ignore the label name
            self.attributeList.append(name)
        for line in file:
            example = Example()
            valueList = line.strip().split()
            if valueList[-1].lower() == 'yes':
                example.setLabel(True)
            else:
                example.setLabel(False)
            # build label list of tree
            self.labels.append(example.label)

            for i, attribute in enumerate(self.attributeList):
                example.addAtribute(attribute=attribute, value=valueList[i])
                if attribute not in self.valueListDict:
                    self.valueListDict[attribute] = [valueList[i]]
                else:
                    if valueList[i] not in self.valueListDict[attribute]:
                        self.valueListDict[attribute].append(valueList[i])
                        # value list always a SET (distinct value)
            self.examples.append(example)
        print("Number of example:", len(self.examples))

    def train(self):
        self.tree = DTL(current_examples=self.examples,parentExamples=self.examples,attributes=self.attributeList,valueListDict=self.valueListDict)


**Making calculation**

In [0]:
'''
for every attribute/feature:
       1.H(S) Calculate the entropy of current state S (example set)
       2.H(S,A_i) Calculate Remainder (entropy with respect to the attribute A_i)
       3.IG(S,A_i) = H(S) - H(S,A_i) : IG for current attribute
'''


def IG(attribute, examples, values_of_this_att):
    '''
        1. Each possible value in current attribute, filtrate examples whose Attribute=value (ie Outlook=sunny,...)
        2. Dictionary : value --> set of corresponding examples
            (ex Attr = Outlook, Dict:{sunny=[ex1,ex2,ex9,ex12], rain=[ex3,ex5], ...}
        3. Calculate IG for current attribute on filtrated examples
        '''
    ig = 0
    value_exampleDict = {} # each value in attribute ---> set of its examples

    '''-----------------Step 1,2'''
    for value in values_of_this_att: #valuelist is distinct values
        sub_examples = []
        for example in examples:
            # an example {Outlook=[sunny],Temp=[hot],Hum=[high],wind=[week],play=[no]}
            if example.attributeDict[attribute] == value:
                sub_examples.append(example)
        value_exampleDict[value] = sub_examples
    '''-----------------Step 3'''
    remainder = remainder_calculate(value_exampleDict, examples)
    entropy = entropy_calculate(examples)
    ig = entropy - remainder
    return ig
  
  
def remainder_calculate(exampleDict, examples):
  '''
  :param exampleDict: value ----> set of examples
  :param examples: current state of example set (S)
  :return: H(S,A = value)
  '''
  remainder = 0
  for value in exampleDict:
      probability = len(exampleDict[value])/len(examples)
      entropy = entropy_calculate(exampleDict[value])
      remainder += probability*entropy
  return remainder


def entropy_calculate(examples):
    '''
    :param examples: set of examples, p_i = (n_i/(pos+neg))
    :return: entropy value of current example set I(+,-) = -sum(p*log2(p))
    '''
    positive,negative = count_label(examples)
    entropyBit = 0
    if positive*negative == 0:
        return 0
    probability = [positive/len(examples), negative/len(examples)]
    for p in probability:
        entropyBit += p*math.log2(p)
    return (-1)*entropyBit


def count_label(examples):
    '''
    :param examples: example set
    :return: number of positive & negative example
    '''
    positive = 0
    negative = 0
    for example in examples:
        if example.label:
            positive += 1
        else:
            negative += 1
    return positive, negative


def majority_label(examples): # Majority==binary classifier; Plurality when it comes to Multi-classifier

    label_list = []
    for example in examples:
        label_list.append(example.label)
    data = Counter(label_list)
    return data.most_common(1)[0][0]


def print_tree(node, level):
    for child in node.children:
        if level > 0:
            print("|", end="")
        for i in range(level):
            print("\t", end="")

        if type(child[1]) == bool:
            print(node.root, "=", child[0] + ":", str(child[1]))
        else:
            print(node.root, "=", child[0])
            print_tree(child[1], level + 1)



**Main method**

In [0]:
def main():
    dt = DecisionTree()
    file = open('bar.txt')
    dt.load_data(file)
    dt.train()
    print("------------ DECISION TREE --------------\n")
    print_tree(dt.tree, 0)

**Dataset 1: bar.txt**

```
alt	bar	fri	hun pat price rain res type est wait
yes	no	no	yes	some	3	no	yes	French	10	yes
yes no  no  yes full    1   no  no  Thai    30  no
no  yes no  no  some    1   no  no  Burger  10  yes
yes no  yes yes full    1   yes no  Thai    30  yes
yes no  yes no  full    3   no  yes French  60 no
no  yes no  yes some    2   yes yes Italian 10  yes
no  yes no  no  none    1   yes no  Burger  10  no
no  no  no  yes some    2   yes yes Thai    10  yes
no  yes yes no  full    1   yes no  Burger  60  no
yes yes yes yes full    3   no  yes Italian 30  no
no  no  no  no  none    1   no  no  Thai    10  no
yes yes yes yes full    1   no  no  Burger  30  yes
```



**Output Tree**


```
pat = some: True
pat = full
|	hun = yes
|		type = French: False
|		type = Thai
|			fri = no: False
|			fri = yes: True
|		type = Burger: True
|		type = Italian: False
|	hun = no: False
pat = none: False
```



**Dataset 2: Play tennis (tennis.txt)**


```
outlook	temperature	humidity	wind	playtennis
sunny	hot	high	weak	no
sunny	hot	high	strong	no
overcast	hot	high	weak	yes
rain	mild	high	weak	yes
rain	cool	normal	weak	yes
rain	cool	normal	strong	no
overcast	cool	normal	strong	yes
sunny	mild	high	weak	no
sunny	cool	normal	weak	yes
rain	mild	normal	weak	yes
sunny	mild	normal	strong	yes
overcast	mild	high	strong	yes
overcast	hot	normal	weak	yes
rain	mild	high	strong	no
```



**Output tree:**


```
outlook = sunny
|	humidity = high: False
|	humidity = normal: True
outlook = overcast: True
outlook = rain
|	wind = weak: True
|	wind = strong: False
```



# *Using Skitlearn on same dataset & entropy criteria*

In [0]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
import pydot
from io import StringIO
import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

df = pd.read_csv("tennis.csv", delimiter="\t")
#df = pd.read_csv("bar.csv", delimiter=",")

attributeNames = [v for v in df.head(0)]

className = attributeNames.pop(-1)
features = attributeNames


dtree = DecisionTreeClassifier(criterion = "entropy")

X = pd.get_dummies(df[attributeNames],drop_first=True)
Y = df[className]
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
dtree.fit(X, Y)
dotfile = StringIO()
tree.export_graphviz(dtree, out_file=dotfile)
(graph,) = pydot.graph_from_dot_data(dotfile.getvalue())
graph.write_png("dtree.png")

**Dataset: bar.csv**


```
alt,bar,fri,hun,pat,price,rain,res,type,est,wait
yes,no,no,yes,some,3,no,yes,French,10,yes
yes,no,no,yes,full,1,no,no,Thai,30,no
no,yes,no,no,some,1,no,no,Burger,10,yes
yes,no,yes,yes,full,1,yes,no,Thai,30,yes
yes,no,yes,no,full,3,no,yes,French,60,no
no,yes,no,yes,some,2,yes,yes,Italian,10,yes
no,yes,no,no,none,1,yes,no,Burger,10,no
no,no,no,yes,some,2,yes,yes,Thai,10,yes
no,yes,yes,no,full,1,yes,no,Burger,60,no
yes,yes,yes,yes,full,3,no,yes,Italian,30,no
no,no,no,no,none,1,no,no,Thai,10,no
yes,yes,yes,yes,full,1,no,no,Burger,30,yes

```



**Output tree:**
<a href="https://ibb.co/fbxOyV"><img src="https://image.ibb.co/jyJ7QA/dtree.png" alt="dtree" border="0"></a><br /><a target='_blank' href='https://aluminumsulfate.net/aluminum-oxide'>Bar DT</a><br />

**Dataset 2: Play tennis tennis.csv**
<a href="https://ibb.co/fr8VdV"><img src="https://preview.ibb.co/dJYwJV/dtree.png" alt="dtree" border="0"></a><br /><a target='_blank' href='https://aluminumsulfate.net/aluminum-oxide'>Play Tennis DT</a><br />
